# import packages and dataset, define the main function

In [56]:
from datetime import date
import pandas as pd
import numpy as np

In [57]:
re_date = pd.read_excel('Inventory Aging Eg.xlsx', sheet_name='Receipt date') # purchased items
sh_date = pd.read_excel('Inventory Aging Eg.xlsx', sheet_name='Ship date') # sold items
cv = pd.read_excel('Inventory Aging Eg.xlsx', sheet_name='Current Value') # current inventory physical value
rt_qty = pd.read_excel('Inventory Aging Eg.xlsx', sheet_name='Returned Qty') # returned itmes

In [58]:
# define a function to show the Receipt date of each item in the warehouse
def rpt_date(value, repetitions,Qty):
    return np.repeat(value, repetitions).iloc[Qty:].tolist()

# Data cleansing

### Find SKUs with positive quantity left in the inventory

In [59]:
# add returned items into purchased items table
re_date = pd.concat([re_date, rt_qty])

In [60]:
# each product sort by date
re_date = re_date.sort_values(['ProductID', 'Received Date'],ascending=[True, True])

In [61]:
re_date

,ProductID,Received Date,QtyReceived
1,2/2.4AMP-CUBES-WHITE,2020-07-28,2100
0,2/2.4AMP-CUBES-WHITE,2023-06-23,500
47512,2/2.4AMP-CUBES-WHITE,2023-10-23,4
39042,2/2.4AMP-CUBES-WHITE,2023-11-02,5
46632,2/2.4AMP-CUBES-WHITE,2023-11-06,2
...,...,...,...
26849,iphone.6-64gb-gold,2019-07-24,1
31807,iphone.6-64gb-gold,2019-08-29,1
39937,‎MGXC2LL/A-16GB-512GB-SILVER,2022-06-21,1
59718,‎SM-T970-256GB-BLACK-NW,2022-06-22,50


In [62]:
# calculate total qty in re_date, group by productID
total_qty_re = re_date.groupby('ProductID')['QtyReceived'].sum()

In [63]:
# Calculate the purchased quantity for each SKU
re_summary = pd.DataFrame(columns=['ProductID', 're_qty'])
re_summary['ProductID'] = total_qty_re.index
re_summary['re_qty'] = total_qty_re.values
print(re_summary)

                             ProductID  re_qty
0                 2/2.4AMP-CUBES-WHITE   11855
1                              38MM-SP     535
2                                 3D-X    6390
3                            3D-XS.MAX     900
4                              40MM-SP    2666
...                                ...     ...
6019  ipad pro-10.5-512gb-spacegray-k1       1
6020                iphone.6-64gb-gold      30
6021      ‎MGXC2LL/A-16GB-512GB-SILVER       1
6022           ‎SM-T970-256GB-BLACK-NW      50
6023            ‎SM-T970-256GB-NAVY-NW      39

[6024 rows x 2 columns]


In [64]:
# calculate total qty in sh_date, group by productID
total_qty_sh = sh_date.groupby('ProductID')['Qty'].sum()

In [65]:
# Calculate the sold quantity for each SKU
sh_summary = pd.DataFrame(columns=['ProductID', 'sh_qty'])
sh_summary['ProductID'] = total_qty_sh.index
sh_summary['sh_qty'] = total_qty_sh.values
print(sh_summary)

                        ProductID  sh_qty
0                    190403171862      35
1                    190403216174      27
2                    190403900257       3
3                    190403914834      11
4            2/2.4AMP-CUBES-WHITE    2103
...                           ...     ...
7263  iphone.6s-16gb-spacegray-k1       7
7264       iphone.6s-64gb-rose-k1      10
7265  iphone.se-16gb-spacegray-k1      44
7266    iphone.xs-512gb-spacegray       7
7267               samphonetest-1       1

[7268 rows x 2 columns]


In [66]:
# left join re_summary and sh_summary, left table is re_summary, create a column calculating 're_qty'-'Total Qty'
import pandas as pd
left_joined = pd.merge(re_summary, sh_summary, on='ProductID', how='left')
left_joined['Difference'] = left_joined['re_qty'] - left_joined['sh_qty']
print(left_joined)

                             ProductID  re_qty  sh_qty  Difference
0                 2/2.4AMP-CUBES-WHITE   11855  2103.0      9752.0
1                              38MM-SP     535     1.0       534.0
2                                 3D-X    6390  1888.0      4502.0
3                            3D-XS.MAX     900  1037.0      -137.0
4                              40MM-SP    2666     4.0      2662.0
...                                ...     ...     ...         ...
6019  ipad pro-10.5-512gb-spacegray-k1       1     1.0         0.0
6020                iphone.6-64gb-gold      30     7.0        23.0
6021      ‎MGXC2LL/A-16GB-512GB-SILVER       1     NaN         NaN
6022           ‎SM-T970-256GB-BLACK-NW      50     NaN         NaN
6023            ‎SM-T970-256GB-NAVY-NW      39     NaN         NaN

[6024 rows x 4 columns]


In [67]:
# take out SKUs which should have negative quantity left
merged_pos = left_joined[left_joined['Difference'] > 0]
print(merged_pos)

                       ProductID  re_qty  sh_qty  Difference
0           2/2.4AMP-CUBES-WHITE   11855  2103.0      9752.0
1                        38MM-SP     535     1.0       534.0
2                           3D-X    6390  1888.0      4502.0
4                        40MM-SP    2666     4.0      2662.0
7                        44MM-SP    3104     6.0      3098.0
...                          ...     ...     ...         ...
6008  Z0UU1LL/A-8GB-512GB-SILVER      39    37.0         2.0
6012         ZOUU1LL/A-8GB-512GB      32     2.0        30.0
6013  ZOUU1LL/A-8GB-512GB-SILVER      38    26.0        12.0
6016                 g900a-black      22     2.0        20.0
6020          iphone.6-64gb-gold      30     7.0        23.0

[1958 rows x 4 columns]


In [68]:
# save negative results in a data frame
merged_neg = left_joined[left_joined['Difference'] <= 0]
print(merged_neg)

                             ProductID  re_qty  sh_qty  Difference
3                            3D-XS.MAX     900  1037.0      -137.0
10                             9H-678P     851   902.0       -51.0
12                             9H-XMAX     250   250.0         0.0
13                               9H-XR    1220  1234.0       -14.0
14                               9H.11    2300  2311.0       -11.0
...                                ...     ...     ...         ...
6009     Z0UU1LL/A-8GB-512GB-SILVER-K1      11    11.0         0.0
6011        ZOUU1LL/A-8GB-128GB-SILVER      40    40.0         0.0
6014     ZOUU1LL/A-8GB-512GB-SILVER-K1       4     6.0        -2.0
6015                  ZT581KL-BLACK-K1       1     1.0         0.0
6019  ipad pro-10.5-512gb-spacegray-k1       1     1.0         0.0

[2504 rows x 4 columns]


In [69]:
# in re_date and sh_date, delete rows where ProductID is not in merged_pos
re_date_pos = re_date[re_date['ProductID'].isin(merged_pos['ProductID'])]
sh_date_pos = sh_date[sh_date['ProductID'].isin(merged_pos['ProductID'])]

In [70]:
import copy
re_date_og = copy.deepcopy(re_date)
sh_date_og = copy.deepcopy(sh_date)

In [71]:
# drop rows in re_date_pos where QtyReceived =0
re_date_pos = re_date_pos[re_date_pos['QtyReceived'] != 0]
sh_date_pos = sh_date_pos[sh_date_pos['Qty'] != 0]

In [72]:
# reset index for both dataframes
sh_date_pos = sh_date_pos.reset_index(drop=True)
re_date_pos = re_date_pos.reset_index(drop=True)

In [73]:
# rename columns in re_date_pos
re_date_pos.rename(columns={'Received Date': 'Ship Date', 'QtyReceived': 'Qty'}, inplace=True)

### Take the negative quantities from receive table to ship table

In [74]:
# for each row in re_date_pos, if QtyReceived <0, delete it from re_date_pos and add to sh_date_pos

for index, row in re_date_pos.iterrows():
  if row['Qty'] < 0:
    re_date_pos.drop(index, inplace=True)
    sh_date_pos.loc[len(sh_date_pos)] = row


In [75]:
sh_date_pos['Qty'] = abs(sh_date_pos['Qty'])

In [76]:
# calculate total qty in sh_date_pos, group by productID
total_qty_sh_pos = sh_date_pos.groupby('ProductID')['Qty'].sum()

### Summarize the new ship table

In [77]:
# Calculate the sold quantity for each SKU
sh_pos_summary = pd.DataFrame(columns=['ProductID', 'sh_qty'])
sh_pos_summary['ProductID'] = total_qty_sh_pos.index
sh_pos_summary['sh_qty'] = total_qty_sh_pos.values
print(sh_pos_summary)

                       ProductID  sh_qty
0           2/2.4AMP-CUBES-WHITE    2103
1                        38MM-SP       1
2                           3D-X    1888
3                        40MM-SP       4
4                        44MM-SP       6
...                          ...     ...
1953  Z0UU1LL/A-8GB-512GB-SILVER      37
1954         ZOUU1LL/A-8GB-512GB       2
1955  ZOUU1LL/A-8GB-512GB-SILVER      26
1956                 g900a-black       2
1957          iphone.6-64gb-gold       7

[1958 rows x 2 columns]


# Run the main function and format the results

In [78]:
df_age = pd.DataFrame(columns=['ProductID', 'Qty', 'Date Difference'])

for i in re_date_pos['ProductID'].unique().tolist():
  re_date_filtered = re_date_pos[re_date_pos['ProductID'] == i].sort_values(['Ship Date'],ascending=True)
  received_dates = re_date_filtered['Ship Date']
  received_quantities = re_date_filtered['Qty']
  if i in sh_pos_summary['ProductID'].tolist():
    qty = sh_pos_summary[sh_pos_summary['ProductID'] == i]['sh_qty'].tolist()[0]
  else:
    qty = 0
  dates = rpt_date(received_dates.astype(str), received_quantities, qty)
  today = date.today()
  date_diff_list = []
  left_qty = sum(received_quantities)-qty
  if left_qty >0:
    for j in dates:
      date_obj = date.fromisoformat(j)
      date_diff = today - date_obj
      date_diff_list.append(date_diff.days)
    df_age.loc[len(df_age)] = [i, left_qty, date_diff_list]
print(df_age)

                       ProductID   Qty  \
0           2/2.4AMP-CUBES-WHITE  9752   
1                        38MM-SP   534   
2                           3D-X  4502   
3                        40MM-SP  2662   
4                        44MM-SP  3098   
...                          ...   ...   
1953  Z0UU1LL/A-8GB-512GB-SILVER     2   
1954         ZOUU1LL/A-8GB-512GB    30   
1955  ZOUU1LL/A-8GB-512GB-SILVER    12   
1956                 g900a-black    20   
1957          iphone.6-64gb-gold    23   

                                        Date Difference  
0     [348, 348, 348, 348, 348, 348, 348, 348, 348, ...  
1     [348, 348, 348, 348, 348, 348, 348, 348, 348, ...  
2     [1475, 1475, 1475, 1475, 1475, 1475, 1475, 147...  
3     [348, 348, 348, 348, 348, 348, 348, 348, 348, ...  
4     [348, 348, 348, 348, 348, 348, 348, 348, 348, ...  
...                                                 ...  
1953                                         [663, 663]  
1954  [1071, 1071, 1071, 1071, 

In [79]:
df_age

,ProductID,Qty,Date Difference
0,2/2.4AMP-CUBES-WHITE,9752,"[348, 348, 348, 348, 348, 348, 348, 348, 348, ..."
1,38MM-SP,534,"[348, 348, 348, 348, 348, 348, 348, 348, 348, ..."
2,3D-X,4502,"[1475, 1475, 1475, 1475, 1475, 1475, 1475, 147..."
3,40MM-SP,2662,"[348, 348, 348, 348, 348, 348, 348, 348, 348, ..."
4,44MM-SP,3098,"[348, 348, 348, 348, 348, 348, 348, 348, 348, ..."
...,...,...,...
1953,Z0UU1LL/A-8GB-512GB-SILVER,2,"[663, 663]"
1954,ZOUU1LL/A-8GB-512GB,30,"[1071, 1071, 1071, 1071, 1071, 1071, 1071, 107..."
1955,ZOUU1LL/A-8GB-512GB-SILVER,12,"[1010, 1010, 1010, 1010, 959, 954, 922, 922, 9..."
1956,g900a-black,20,"[1841, 1841, 1841, 1841, 1841, 1841, 1841, 184..."


In [80]:
# convert "Date Deifference" column in df_age from list to string

df_age['Date Difference'] = df_age['Date Difference'].apply(lambda x: ','.join(map(str, x)))


In [81]:
df_age

,ProductID,Qty,Date Difference
0,2/2.4AMP-CUBES-WHITE,9752,"348,348,348,348,348,348,348,348,348,348,348,34..."
1,38MM-SP,534,"348,348,348,348,348,348,348,348,348,348,348,34..."
2,3D-X,4502,"1475,1475,1475,1475,1475,1475,1475,1475,1475,1..."
3,40MM-SP,2662,"348,348,348,348,348,348,348,348,348,348,348,34..."
4,44MM-SP,3098,"348,348,348,348,348,348,348,348,348,348,348,34..."
...,...,...,...
1953,Z0UU1LL/A-8GB-512GB-SILVER,2,"663,663"
1954,ZOUU1LL/A-8GB-512GB,30,"1071,1071,1071,1071,1071,1071,1071,1071,1071,1..."
1955,ZOUU1LL/A-8GB-512GB-SILVER,12,"1010,1010,1010,1010,959,954,922,922,912,818,81..."
1956,g900a-black,20,"1841,1841,1841,1841,1841,1841,1841,1841,1841,1..."


In [82]:
# add four columns "1-30", "31-60","61-90", ">90" to df_age, fill these four columns with count of values from "Date Difference" column for each productID

import re

def categorize_date_diff(date_diff):
  if date_diff <= 30:
    return "Aged 1-30"
  elif date_diff <= 60:
    return "Aged 31-60"
  elif date_diff <= 90:
    return "Aged 61-90"
  elif date_diff <=365:
    return "Aged 91-365"
  else:
    return "Aged >365"

df_age["Aged 1-30"] = 0
df_age["Aged 31-60"] = 0
df_age["Aged 61-90"] = 0
df_age["Aged 91-365"] = 0
df_age["Aged >365"] = 0

for i, row in df_age.iterrows():
  date_diffs = [int(x) for x in row["Date Difference"].split(",")]
  for date_diff in date_diffs:
    category = categorize_date_diff(date_diff)
    df_age.at[i, category] += 1

df_age.drop(columns=["Date Difference"], inplace=True)

In [83]:
df_age

,ProductID,Qty,Aged 1-30,Aged 31-60,Aged 61-90,Aged 91-365,Aged >365
0,2/2.4AMP-CUBES-WHITE,9752,57,20,70,9605,0
1,38MM-SP,534,7,0,12,515,0
2,3D-X,4502,0,0,0,0,4502
3,40MM-SP,2662,19,11,17,2615,0
4,44MM-SP,3098,11,7,20,3060,0
...,...,...,...,...,...,...,...
1953,Z0UU1LL/A-8GB-512GB-SILVER,2,0,0,0,0,2
1954,ZOUU1LL/A-8GB-512GB,30,0,0,0,1,29
1955,ZOUU1LL/A-8GB-512GB-SILVER,12,0,0,0,1,11
1956,g900a-black,20,0,0,0,0,20


# Add negative quantities (removed at the beginning) and the current available qty to results

In [91]:
# add ProductID and Differnece columns from merged_neg to the ProductID and Qty columns in df_age, value for other columns are 0
df_total = pd.concat([df_age,pd.DataFrame({'ProductID': merged_neg['ProductID'].tolist(), 'Qty': merged_neg['Difference'].tolist(), 'Aged 1-30': 0, 'Aged 31-60': 0, 'Aged 61-90': 0, 'Aged 91-365': 0, 'Aged >365': 0})])


In [92]:
df_total=df_total.reset_index(drop=True)

In [93]:
df_total

,ProductID,Qty,Aged 1-30,Aged 31-60,Aged 61-90,Aged 91-365,Aged >365
0,2/2.4AMP-CUBES-WHITE,9752.0,57,20,70,9605,0
1,38MM-SP,534.0,7,0,12,515,0
2,3D-X,4502.0,0,0,0,0,4502
3,40MM-SP,2662.0,19,11,17,2615,0
4,44MM-SP,3098.0,11,7,20,3060,0
...,...,...,...,...,...,...,...
4457,Z0UU1LL/A-8GB-512GB-SILVER-K1,0.0,0,0,0,0,0
4458,ZOUU1LL/A-8GB-128GB-SILVER,0.0,0,0,0,0,0
4459,ZOUU1LL/A-8GB-512GB-SILVER-K1,-2.0,0,0,0,0,0
4460,ZT581KL-BLACK-K1,0.0,0,0,0,0,0


In [94]:
# add current value to df_total
df_total_2 = pd.merge(df_total, cv, left_on='ProductID', right_on='ProductName', how='left')

In [95]:
# move Available Qty column before Qty column and drop useless columns
df_total_2 = df_total_2[['ProductID', 'Available Qty', 'Qty', 'Aged 1-30', 'Aged 31-60', 'Aged 61-90', 'Aged 91-365', 'Aged >365']]

In [96]:
df_total_2

,ProductID,Available Qty,Qty,Aged 1-30,Aged 31-60,Aged 61-90,Aged 91-365,Aged >365
0,2/2.4AMP-CUBES-WHITE,NaN,9752.0,57,20,70,9605,0
1,38MM-SP,1060.0,534.0,7,0,12,515,0
2,3D-X,0.0,4502.0,0,0,0,0,4502
3,40MM-SP,933.0,2662.0,19,11,17,2615,0
4,44MM-SP,1939.0,3098.0,11,7,20,3060,0
...,...,...,...,...,...,...,...,...
4457,Z0UU1LL/A-8GB-512GB-SILVER-K1,0.0,0.0,0,0,0,0,0
4458,ZOUU1LL/A-8GB-128GB-SILVER,0.0,0.0,0,0,0,0,0
4459,ZOUU1LL/A-8GB-512GB-SILVER-K1,0.0,-2.0,0,0,0,0,0
4460,ZT581KL-BLACK-K1,0.0,0.0,0,0,0,0,0


In [97]:
df_total_2.to_csv('Inventory Age.csv')